In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [3]:
#2. 문자 집합 만들기
world_set = list(set(sentence))

In [4]:
vocab = {char : i for i, char in enumerate(world_set)}

In [5]:
print(vocab)

{'c': 0, 't': 1, 's': 2, 'i': 3, 'g': 4, 'o': 5, 'd': 6, 'f': 7, '.': 8, 'k': 9, 'm': 10, 'e': 11, ',': 12, 'a': 13, 'h': 14, 'B': 15, 'n': 16, ' ': 17, 'y': 18, 'b': 19, 'l': 20, 'u': 21, 'w': 22, 'p': 23, 'r': 24, "'": 25}


In [6]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [7]:
#4. 하이퍼 파라미터 설정 - 자유롭게

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [9]:
# 출력해서 한 칸씩 쉬프트된 것 확인

print(x_data[0])
print(y_data[0])

[15, 24, 3, 0, 9, 17, 22, 13, 20, 20]
[24, 3, 0, 9, 17, 22, 13, 20, 20, 2]


In [10]:
##6. 입력 시퀀스 원핫인코딩 수행.


## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

<ipython-input-10-17bdfe0520bd>:10: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  X = torch.FloatTensor(x_one_hot)


In [11]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))


훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [12]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [13]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([24,  3,  0,  9, 17, 22, 13, 20, 20,  2])


In [14]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first = True)
    self.fc = torch.nn.Linear(hidden_dim, vocab_size) #output_dim = 26

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [15]:
net = Net(vocab_size, hidden_size, 2)

In [16]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [17]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [18]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

dhdpppdppdddppdppddpdpppddpppdpddpddpdppddppdddpdsdppddpddpdppppddpdpdpppdppdddppdpppppddpdpppdddpdpdddsdpdddppddppddpddpppdddddppppdppppddpdpdpppdppdddppdppddpdpppdddsddddddsdddpddppddpdpppppddpdd
aasswaasaawwwwsaswwwswwwwasawss wswaaaaawwawswawawwwawsawasawawswwwassawaasaawwwwsswawawwwswswwwwwwwsswwswwaasswwsaawwwsswswswwwsaasawawswwwassawaasaawwwwsaswwwswswwwawwwawawwwwasasawwswasaswawsaww
a              a     a   a    a  aa     a      a   a   a    a   a a   a           a        a  aa   a       a         a  a  a         a   a a   a           a     a  a  aaa     aa     a  a    a   a  
a                                                                                                                                                                                                    
                                                                                                                                                                                                     
          

In [19]:
predict_str # 끝에가 좀 걸림....

"rick walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thingsu"

In [28]:
sentence = ("Give me a cigarette"
            "The weather is perfect for dying. "
            "Jaseong, save me. I don't want to die.")

In [29]:
world_set = list(set(sentence))

In [30]:
vocab = {char: idx for idx, char in enumerate(world_set)}

In [31]:
print(vocab)

{'c': 0, 't': 1, 'I': 2, 'G': 3, 's': 4, 'i': 5, 'g': 6, 'o': 7, 'd': 8, 'f': 9, '.': 10, 'm': 11, 'e': 12, 'v': 13, ',': 14, 'a': 15, 'h': 16, 'y': 17, 'n': 18, ' ': 19, 'w': 20, 'T': 21, 'J': 22, 'p': 23, 'r': 24, "'": 25}


In [32]:
len(vocab)

26

In [33]:
hidden_size = vocab_size
sequence_length = 15
learning_rate = 0.01


#5. seqence 길이 단위 자르기
# 데이터 구성을 위한 리스트
x_data = []
y_data = []

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : i + sequence_length]
  y_str = sentence[i+1 : i+1 + sequence_length]
  print(i, x_str, "->", y_str)
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])
print(x_data[0])
print(y_data[0])

0 Give me a cigar -> ive me a cigare
1 ive me a cigare -> ve me a cigaret
2 ve me a cigaret -> e me a cigarett
3 e me a cigarett ->  me a cigarette
4  me a cigarette -> me a cigaretteT
5 me a cigaretteT -> e a cigaretteTh
6 e a cigaretteTh ->  a cigaretteThe
7  a cigaretteThe -> a cigaretteThe 
8 a cigaretteThe  ->  cigaretteThe w
9  cigaretteThe w -> cigaretteThe we
10 cigaretteThe we -> igaretteThe wea
11 igaretteThe wea -> garetteThe weat
12 garetteThe weat -> aretteThe weath
13 aretteThe weath -> retteThe weathe
14 retteThe weathe -> etteThe weather
15 etteThe weather -> tteThe weather 
16 tteThe weather  -> teThe weather i
17 teThe weather i -> eThe weather is
18 eThe weather is -> The weather is 
19 The weather is  -> he weather is p
20 he weather is p -> e weather is pe
21 e weather is pe ->  weather is per
22  weather is per -> weather is perf
23 weather is perf -> eather is perfe
24 eather is perfe -> ather is perfec
25 ather is perfec -> ther is perfect
26 ther is perfect -> 

In [34]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환
x_one_hot = np.array(x_one_hot)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([76, 15, 26])
레이블의 크기 : torch.Size([76, 15])


In [42]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [43]:
net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [44]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([76, 15, 26])


In [45]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

occoocooocoooooooccooooooocIoocoooooooooocooooooooooicIoooooooooooocoocccoooiocoooocoococo
ooeooeooeoo oeoooeooerooooeeooeoooeooeooooeooooooeeeeeooooooeoooeooeooeoooooooeoooeoeooo o
oee  e                 e   e         e    e   e                    e           e          
eee  e                                                                                    
e    e                                                             e                      
e    e                                                             e                      
e    e                                                             e                      
e                                                                                         
e                                                                                         
e    e                 e                                           e                      
e e  e  e          te  etteet e   e  e   se   e      e       sete te   ete     et     te  

In [46]:
predict_str

"ive me a cigaretteThe weather is perfect for dying. Jaseong, save me. I don't want to die."

In [47]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn1 = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.rnn2 = torch.nn.RNN(input_size=hidden_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        out, _ = self.rnn1(x)
        out, _ = self.rnn2(out)
        out = self.fc(out)
        return out

In [51]:
net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [52]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

mmm     m    mm     m            m    m       m             m   m     m          m        
                                                                                          
                                                                                          
                                                                                          
  e  e  e e  ee  e  e  e  e   e  ee     ee  e e   e ee e  e  e  e  e  e  e  e e  e ee  e  
  e  e  e     e  e  e  e  e   e  ee ee  ee  e e   e ee e  e  e  e  e  e  e  e ee e  e  e  
  e  e  e  e  e  e  e  e  e  e   ee ee      e e  ee ee e  e  e  e  e  e  e  e ee e  e  e  
  e  e  e  e  e  e  e  e  e  e   ee ee e   ee e  ee ee e  e  e  e  e  e  e  e ee e  e  e  
  e  e  e  e  e  e  e  e  e  e    e ee e    e e  ee e  e  e  e  e  e  e  e  e ee e  e  e  
  e  e  e     e  e  e  e  e  e e ge e  e e  e e  ee ee e  e  e  e  e  e  e  e ee e  e  e  
  e  e  e   e e  e  e  e  e    e ge ee   e  e e   e ee e  e  eene  e  e ne  e ge e te  e  

In [53]:
predict_str

"ive me a cigaretteThe weather is perfect for dying. Jaseong, save me. I don't want to die."

In [54]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [55]:
net = Net(vocab_size, hidden_size, 4)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [56]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

..ha h  a h h    a       h                  h   h              h   h     h h     h     h  
a                                                                                         
a                                                                                         
                                                                                          
                                                                                          
                                                                                          
  e ee  e  e ee e   e ee ee  ee   e ee e  e  e   e  e  e e e e  e ee  e     e e  e e   e  
 ee ee ee ee ee ee ee ee ee  eee ee ee ee e eee ee  e ee e e e ee ee ee eee e eeee ee ee  
 ee ee ee  e ee ee ee ee ee  eee ee ee ee e eee ee  e  e e e e ee ee ee eee e ee e ee ee  
  e ee ee  e ee ee ee te ee  ee  ee ee ee e  e  ee  e  e e t e  e ee ee te  e te e te  e  
  e te  e  e te te  e te te  ee  ee te te e  e  ee  e te t m m te te  e te  e t  e te  e  

In [57]:
print(predict_str)

ive me a cigaretteThe weather is perfect for dying. Jaseong, save me. I don't want to die.


In [58]:
hidden_size = vocab_size
sequence_length = 10
learning_rate = 0.001

In [59]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_size=input_dim, hidden_size=hidden_dim, num_layers=layers, batch_first=True)
    self.fc = torch.nn.Linear(in_features=hidden_dim, out_features=vocab_size)

  def forward(self, x):
    x, _ = self.rnn(x)
    x = self.fc(x)
    return x

In [60]:
net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [62]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([world_set[t] for t in result])
        else:
            predict_str += world_set[result[-1]]

    print(predict_str)

wihimimiwimimiwiiiimiwimiiiiiiiwmiwiiiimiiwimiwiiiwmiwiwiiiwiiwiiwiiiiiwiwiiiiimiiiiwimwii
wwhimimiwimwmiwwhiimiwimiiiiiiiwmiwiiiimiiwwmiwiiiwmiwiwiiiwiiwiiwiiiiiwiwiiiiimiwiimimwmi
wwhimimiwimwmiwwhiimiwimiiiiimiwmiwiiiimiimwmiwiiiwmiwiwiiiwmiwiiwiihiiwiwwiiiimiwiimimwmi
wwmimimiwimwmiwwhiimiwimiwiimmiwmiwmmiimiimwmiwiimwmiwiwwiiwmiwiiwiimiiwiwwmiiimiwhimimwmi
wwmimimi imwmimwhiimiwimiwiimmiwmiwmmiimiwmwmiwiimwmiwiwwimwmw ii iimimwiwwmihimiwhi imwmi
wwmimimi mmwmwmwhiemi imiwii miwmiwmmimmiwmwmiwiimwmiwiwwimwmw wi iimmmwiwwmihimiwhi imwmi
wwmimimi mmwmwm hhemi imimie miwmiwmmimmim wmi emmwmi iwwmmmmw wi iimemwi wmi imiwei imwme
wwm mimi mmmmwm heemi eme ie mm mi  m emim wmi em wmi i  mmmmw we mieemwi  me imimei wm me
wwe mimi mm m m ee mi eme ee mm  i  m emim wmi em wmi i  memmw  e meeem i  me im  ee wm me
wwe me e mm m   ee me em  ee mm  i  m em m  mi em   i e  mem w  e e  em i  ee e   ee  m me
wwe me e m      ee me e   ee  m  e  m em m  mi em   i e  m m e  e e   m m  ee e   ee  m e 

In [63]:
hidden_size = vocab_size
sequence_length = 10
learning_rate = 0.1

In [64]:
net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [65]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([world_set[t] for t in result])
        else:
            predict_str += world_set[result[-1]]

    print(predict_str)

vvvvvyvvvvvvyvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvyvyvvvvyvyyvyvvvyvyyvvvvyvvvvvvvvvvvvvvvvyvvvvv
        o o         o      o         oo  o       o        o        o  o    oo             
eeeeee eeeeeeeeeeeeeeeeeaeee eeaeeeeee  ea  eee eee eneeeege eneeeee eeeeeeet enee eeeeeee
agG,cc  gGr g, ggGchgc agG gG ggGragrh'tggG agGrhg, agGrhgGr ggc gg,r gG ngGc hgGrhgg, gGc
agg..   anGG.     e     na. .    a           a  c      an...   naa..    sn ..    aa.     a
i  aJtr ac dgtra yftc.wcnttt wsncoa  ay  f coacttf  cocotat csacotat wcoco gt coag c  coag
i waa  wsr da  dod wf wdot atwsadodJ wa  Jarwdo  dt wad  aat ssi waa wsodoat wIodc dar do 
i wsa  facadatea   hg wsst gt saiaar dst I  adatngt Iavrongt svirodetwIadatef waar d d at 
i watehh Iaeateat hh  wIat f  aaeaaefeet f  amaingt Jaisa gt Jaisan t IaIangt waoeoe net  
iaeane.hedaeaheioethe weat e  aa mar eetefaeamaing. Jaesa gt Jasrame. eadang. waeete nene.
ioe me  esieaneate.he  eather ae mer eet eoe mying. Jaseone. saseome. e done. wanete neset

In [66]:
  print(predict_str)

ive me a cigaretteThe weather is perfect for dying. Jaseong, save me. I don't want to die.


In [67]:
hidden_size = vocab_size
sequence_length = 5
learning_rate = 0.01

In [68]:
net = Net(vocab_size, hidden_size, 2)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [69]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)

    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([world_set[t] for t in result])
        else:
            predict_str += world_set[result[-1]]

    print(predict_str)

ttt tst sttt stttc tt tstttsttttcstt tt t ttttttsttstss t t ststtttst ttIttttcsstttstststt
ttttttttttttttttt  tttttttt tttt tttttttttttttttttttttt ttttttttttttttttttttt tttttttttttt
t t tttttttt tttt  tt ttttt t tt ttt ttt  tttttt tt ttt ttt  t tt t t ttttttt  tttttttttt 
                                          t                    t         t      t         
                                                                                          
                                                                                          
     e   e    e     e  e  e       e  e            e        e    e  e    e              e  
     e   e    e     e  e  e   e   e  e    e  e    e  e e   e e  e  e    e      e       e  
     e  ee    e  e  e  e  e   e   e  e    e  ee   e  e e   e ee e  e   ee   e  e       e  
     e  ee    e  e  e  e  e   ee  e  e   ee  ee   e  e e   e ee e  e  eee   e  e    e  e  
     e  ee    e  e te  e  e   ee  e  e   ee  ee   e  e e   e ee e  e  eee   e  e    e  e  

In [70]:
# G 네 이놈.....